In [19]:
import warnings
warnings.filterwarnings('ignore')

### 파일 불러오기

In [3]:
import pandas as pd

mons = pd.read_csv("data/역별월별비율_15_22.csv")
days = pd.read_csv("data/0913_최종역별요일별비율.csv")

In [5]:
pip install xlrd>=2.0.1

Note: you may need to restart the kernel to use updated packages.


In [6]:
# 엑셀 파일에서 모든 시트를 읽어서 딕셔너리 형태로 저장
excel_file = pd.ExcelFile('raw/서울교통공사 역별 혼잡도(11년).xls')
sheet_names = excel_file.sheet_names  # 시트 이름 목록
sheet_names

['평일 상선', '평일 하선', '토요일 상선', '토요일 하선', '일요일 상선', '일요일 하선']

In [7]:
# 모든 시트의 데이터를 저장할 빈 리스트를 생성
all_dfs = []

for sheet_name in sheet_names:
    # 필요한 행부터 읽기 시작
    df = pd.read_excel(excel_file, sheet_name=sheet_name, header=2)

    # 불필요한 열 삭제
    df.dropna(axis=1, how='all', inplace=True)

    # 요일구분 컬럼 추가
    day_of_week = sheet_name.split()[-2]  # 시트 이름에서 요일 정보 추출
    df['요일구분'] = day_of_week

    # 상하구분 컬럼 추가 (상선 또는 하선)
    direction = '상선' if '상선' in sheet_name else '하선'
    df['상하구분'] = direction

    # 데이터프레임을 리스트에 추가
    all_dfs.append(df)

In [8]:
all_dfs

[      호선       역명  방향  5시대  6시대  7시대  8시대  9시대  10시대  11시대  ...  17시대  18시대  \
 0    1호선    서울역.1  상선    8   16   44   76   54    38    40  ...    34    49   
 1    NaN     시청.1  상선    8   16   39   69   55    43    45  ...    42    68   
 2    NaN       종각  상선    9   13   28   53   45    40    41  ...    45    79   
 3    NaN   종로3가.1  상선    9   13   31   57   49    45    47  ...    56    97   
 4    NaN     종로5가  상선    8   12   26   46   43    40    42  ...    58   102   
 ..   ...      ...  ..  ...  ...  ...  ...  ...   ...   ...  ...   ...   ...   
 118  NaN     이촌.4  상선   15   23   44   77   44    35    37  ...    39    38   
 119  NaN       동작  상선   15   25   48   73   46    31    40  ...    39    38   
 120  NaN  총신대입구.4  상선   16   27   50   79   42    32    40  ...    37    38   
 121  NaN     사당.4  상선   20   25   51   77   45    33    40  ...    39    40   
 122  NaN      남태령  상선   21   36  107  135   93    59    66  ...    60    74   
 
      19시대  20시대  21시대  22시대  23시대  24

In [9]:
# 리스트에 있는 모든 데이터프레임을 합치기
df_13 = pd.concat(all_dfs, ignore_index=True)

In [10]:
df_13.info

<bound method DataFrame.info of       호선       역명  방향  5시대  6시대  7시대  8시대  9시대  10시대  11시대  ...  17시대  18시대  \
0    1호선    서울역.1  상선    8   16   44   76   54    38    40  ...    34    49   
1    NaN     시청.1  상선    8   16   39   69   55    43    45  ...    42    68   
2    NaN       종각  상선    9   13   28   53   45    40    41  ...    45    79   
3    NaN   종로3가.1  상선    9   13   31   57   49    45    47  ...    56    97   
4    NaN     종로5가  상선    8   12   26   46   43    40    42  ...    58   102   
..   ...      ...  ..  ...  ...  ...  ...  ...   ...   ...  ...   ...   ...   
733  NaN     이촌.4  하선   27    9    7   12   23    25    24  ...    40    35   
734  NaN       동작  하선    0   11    7   12   24    26    25  ...    40    36   
735  NaN  총신대입구.4  하선    0   12    8   14   27    31    28  ...    40    36   
736  NaN     사당.4  하선   11   12    8   17   30    39    29  ...    34    30   
737  NaN      남태령  하선   11   24   17   26   50    78    55  ...    69    64   

     19시대  20시대  21

In [11]:
# 시간대 컬럼 이름 변경
new_columns = {
    '5시대': '05시~06시', '6시대': '06시~07시', '7시대': '07시~08시', '8시대': '08시~09시', '9시대': '09시~10시',
    '10시대': '10시~11시', '11시대': '11시~12시', '12시대': '12시~13시', '13시대': '13시~14시', '14시대': '14시~15시',
    '15시대': '15시~16시', '16시대': '16시~17시', '17시대': '17시~18시', '18시대': '18시~19시', '19시대': '19시~20시',
    '20시대': '20시~21시', '21시대': '21시~22시', '22시대': '22시~23시', '23시대': '23시~24시', '24시대': '24시~01시'
}

df_13.rename(columns=new_columns, inplace=True)

In [12]:
# 컬럼 순서 변경
new_column_order = ['요일구분', '호선', '역명', '상하구분', '05시~06시', '06시~07시', '07시~08시', '08시~09시', '09시~10시', '10시~11시', '11시~12시',
                    '12시~13시', '13시~14시', '14시~15시', '15시~16시', '16시~17시', '17시~18시', '18시~19시', '19시~20시', '20시~21시',
                    '21시~22시', '22시~23시', '23시~24시', '24시~01시']

In [13]:
df_13 = df_13[new_column_order]

In [14]:
df_13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 738 entries, 0 to 737
Data columns (total 24 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   요일구분     738 non-null    object
 1   호선       24 non-null     object
 2   역명       738 non-null    object
 3   상하구분     738 non-null    object
 4   05시~06시  738 non-null    int64 
 5   06시~07시  738 non-null    int64 
 6   07시~08시  738 non-null    int64 
 7   08시~09시  738 non-null    int64 
 8   09시~10시  738 non-null    int64 
 9   10시~11시  738 non-null    int64 
 10  11시~12시  738 non-null    int64 
 11  12시~13시  738 non-null    int64 
 12  13시~14시  738 non-null    int64 
 13  14시~15시  738 non-null    int64 
 14  15시~16시  738 non-null    int64 
 15  16시~17시  738 non-null    int64 
 16  17시~18시  738 non-null    int64 
 17  18시~19시  738 non-null    int64 
 18  19시~20시  738 non-null    int64 
 19  20시~21시  738 non-null    int64 
 20  21시~22시  738 non-null    int64 
 21  22시~23시  738 non-null    int64 
 22  23

### 결측값 처리 (호선명 채우기)

In [15]:
# 비어있는 호선 값 채우기
for index, row in df_13.iterrows():
    if pd.isnull(row['호선']):  # 호선 값이 NULL인 경우
        df_13.at[index, '호선'] = df_13.at[index - 1, '호선']  # 직전 행의 호선 값 복사

In [16]:
df_13.head()

,요일구분,호선,역명,상하구분,05시~06시,06시~07시,07시~08시,08시~09시,09시~10시,10시~11시,...,15시~16시,16시~17시,17시~18시,18시~19시,19시~20시,20시~21시,21시~22시,22시~23시,23시~24시,24시~01시
0,평일,1호선,서울역.1,상선,8,16,44,76,54,38,...,29,30,34,49,36,26,21,21,12,4
1,평일,1호선,시청.1,상선,8,16,39,69,55,43,...,33,39,42,68,49,38,31,31,20,5
2,평일,1호선,종각,상선,9,13,28,53,45,40,...,36,44,45,79,57,46,41,42,25,7
3,평일,1호선,종로3가.1,상선,9,13,31,57,49,45,...,43,55,56,97,71,57,49,53,31,8
4,평일,1호선,종로5가,상선,8,12,26,46,43,40,...,42,56,58,102,75,59,51,56,32,8


### df_13 데이터 형식 수정

In [26]:
# 시간대 실수형으로 형식 바꾸기
time_columns = ['05시~06시', '06시~07시', '07시~08시', '08시~09시', '09시~10시',
                '10시~11시', '11시~12시', '12시~13시', '13시~14시', '14시~15시',
                '15시~16시', '16시~17시', '17시~18시', '18시~19시', '19시~20시',
                '20시~21시', '21시~22시', '22시~23시', '23시~24시', '24시~01시']

for col in time_columns:
    df_13[col] = df_13[col].astype('float64')

In [27]:
df_13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 738 entries, 0 to 737
Data columns (total 24 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   요일구분     738 non-null    object 
 1   호선       738 non-null    int32  
 2   역명       738 non-null    object 
 3   상하구분     738 non-null    object 
 4   05시~06시  738 non-null    float64
 5   06시~07시  738 non-null    float64
 6   07시~08시  738 non-null    float64
 7   08시~09시  738 non-null    float64
 8   09시~10시  738 non-null    float64
 9   10시~11시  738 non-null    float64
 10  11시~12시  738 non-null    float64
 11  12시~13시  738 non-null    float64
 12  13시~14시  738 non-null    float64
 13  14시~15시  738 non-null    float64
 14  15시~16시  738 non-null    float64
 15  16시~17시  738 non-null    float64
 16  17시~18시  738 non-null    float64
 17  18시~19시  738 non-null    float64
 18  19시~20시  738 non-null    float64
 19  20시~21시  738 non-null    float64
 20  21시~22시  738 non-null    float64
 21  22시~23시  738 non

### df_13 [24사~01시] 결측값 처리
- 주말(토요일, 일요일)에는 24시~01시에 운행을 안함.
- 중앙값인 0으로 결측값 대체

In [28]:
df_13.isna().sum()

요일구분       0
호선         0
역명         0
상하구분       0
05시~06시    0
06시~07시    0
07시~08시    0
08시~09시    0
09시~10시    0
10시~11시    0
11시~12시    0
12시~13시    0
13시~14시    0
14시~15시    0
15시~16시    0
16시~17시    0
17시~18시    0
18시~19시    0
19시~20시    0
20시~21시    0
21시~22시    0
22시~23시    0
23시~24시    0
24시~01시    0
dtype: int64

In [29]:
null_df = df_13[df_13['24시~01시'].isnull()]
null_df.head()

,요일구분,호선,역명,상하구분,05시~06시,06시~07시,07시~08시,08시~09시,09시~10시,10시~11시,...,15시~16시,16시~17시,17시~18시,18시~19시,19시~20시,20시~21시,21시~22시,22시~23시,23시~24시,24시~01시


In [30]:
null_df.요일구분.value_counts()

Series([], Name: count, dtype: int64)

In [31]:
df_13['24시~01시'].median()

0.0

In [32]:
df_13['24시~01시'].fillna(0, inplace=True)

In [33]:
df_13.isna().sum()

요일구분       0
호선         0
역명         0
상하구분       0
05시~06시    0
06시~07시    0
07시~08시    0
08시~09시    0
09시~10시    0
10시~11시    0
11시~12시    0
12시~13시    0
13시~14시    0
14시~15시    0
15시~16시    0
16시~17시    0
17시~18시    0
18시~19시    0
19시~20시    0
20시~21시    0
21시~22시    0
22시~23시    0
23시~24시    0
24시~01시    0
dtype: int64

### 혼잡도 구하기

In [34]:
melt_13 = pd.melt(df_13, id_vars=['요일구분', '호선', '역명', '상하구분'], var_name='시간대', value_name='값')
melt_13

,요일구분,호선,역명,상하구분,시간대,값
0,평일,1,서울역.1,상선,05시~06시,8.0
1,평일,1,시청.1,상선,05시~06시,8.0
2,평일,1,종각,상선,05시~06시,9.0
3,평일,1,종로3가.1,상선,05시~06시,9.0
4,평일,1,종로5가,상선,05시~06시,8.0
...,...,...,...,...,...,...
14755,일요일,4,이촌.4,하선,24시~01시,0.0
14756,일요일,4,동작,하선,24시~01시,0.0
14757,일요일,4,총신대입구.4,하선,24시~01시,0.0
14758,일요일,4,사당.4,하선,24시~01시,4.0


In [35]:
melt_13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14760 entries, 0 to 14759
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   요일구분    14760 non-null  object 
 1   호선      14760 non-null  int32  
 2   역명      14760 non-null  object 
 3   상하구분    14760 non-null  object 
 4   시간대     14760 non-null  object 
 5   값       14760 non-null  float64
dtypes: float64(1), int32(1), object(4)
memory usage: 634.3+ KB


In [36]:
mons.rename(columns={'호선명': '호선', '지하철역': '역명'}, inplace=True)

In [37]:
mons = mons[(mons['호선'] == '1호선') | (mons['호선'] == '2호선') | (mons['호선'] == '3호선') | (mons['호선'] == '4호선')]

In [38]:
mons.호선.value_counts

<bound method IndexOpsMixin.value_counts of 0       1호선
1       1호선
2       1호선
3       1호선
4       1호선
       ... 
1747    4호선
1748    4호선
1749    4호선
1750    4호선
1751    4호선
Name: 호선, Length: 1752, dtype: object>

In [39]:
mons = mons.replace({'호선' : '1호선'}, '1')
mons = mons.replace({'호선' : '2호선'}, '2')
mons = mons.replace({'호선' : '3호선'}, '3')
mons = mons.replace({'호선' : '4호선'}, '4')

In [40]:
mons

,호선,역명,사용월,비율
0,1,동대문,1,0.954919
1,1,동대문,2,0.882286
2,1,동대문,3,1.052501
3,1,동대문,4,1.032322
4,1,동대문,5,1.080913
...,...,...,...,...
1747,4,회현(남대문시장),8,0.947942
1748,4,회현(남대문시장),9,0.941920
1749,4,회현(남대문시장),10,1.053734
1750,4,회현(남대문시장),11,1.051536


In [41]:
melt_13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14760 entries, 0 to 14759
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   요일구분    14760 non-null  object 
 1   호선      14760 non-null  int32  
 2   역명      14760 non-null  object 
 3   상하구분    14760 non-null  object 
 4   시간대     14760 non-null  object 
 5   값       14760 non-null  float64
dtypes: float64(1), int32(1), object(4)
memory usage: 634.3+ KB


In [42]:
melt_13['호선'] = melt_13['호선'].astype(str)
merged_df = melt_13.merge(mons, on=['호선', '역명'], how='inner')
merged_df

,요일구분,호선,역명,상하구분,시간대,값,사용월,비율
0,평일,1,종각,상선,05시~06시,9.0,1,1.026593
1,평일,1,종각,상선,05시~06시,9.0,2,0.897782
2,평일,1,종각,상선,05시~06시,9.0,3,1.027239
3,평일,1,종각,상선,05시~06시,9.0,4,0.987545
4,평일,1,종각,상선,05시~06시,9.0,5,1.014592
...,...,...,...,...,...,...,...,...
99355,일요일,4,남태령,하선,24시~01시,11.0,8,1.193308
99356,일요일,4,남태령,하선,24시~01시,11.0,9,1.096531
99357,일요일,4,남태령,하선,24시~01시,11.0,10,0.997312
99358,일요일,4,남태령,하선,24시~01시,11.0,11,1.014522


In [43]:
merged_df['월혼잡도'] = merged_df['값'] * merged_df['비율']
merged_df

,요일구분,호선,역명,상하구분,시간대,값,사용월,비율,월혼잡도
0,평일,1,종각,상선,05시~06시,9.0,1,1.026593,9.239336
1,평일,1,종각,상선,05시~06시,9.0,2,0.897782,8.080034
2,평일,1,종각,상선,05시~06시,9.0,3,1.027239,9.245150
3,평일,1,종각,상선,05시~06시,9.0,4,0.987545,8.887901
4,평일,1,종각,상선,05시~06시,9.0,5,1.014592,9.131327
...,...,...,...,...,...,...,...,...,...
99355,일요일,4,남태령,하선,24시~01시,11.0,8,1.193308,13.126386
99356,일요일,4,남태령,하선,24시~01시,11.0,9,1.096531,12.061844
99357,일요일,4,남태령,하선,24시~01시,11.0,10,0.997312,10.970433
99358,일요일,4,남태령,하선,24시~01시,11.0,11,1.014522,11.159747


In [44]:
merged_df.역명.value_counts()

역명
종각       1440
금호       1440
매봉       1440
양재       1440
남부터미널    1440
         ... 
용답       1440
신답       1440
용두       1440
도림천      1440
남태령      1440
Name: count, Length: 69, dtype: int64

In [45]:
days.head()

,호선,역명,요일,비율
0,1,동대문,금,1.061534
1,1,동대문,목,1.020850
2,1,동대문,수,1.026641
3,1,동대문,월,1.005251
4,1,동대문,일,0.873151


In [46]:
days.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2001 entries, 0 to 2000
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   호선      2001 non-null   int64  
 1   역명      2001 non-null   object 
 2   요일      2001 non-null   object 
 3   비율      2001 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 62.7+ KB


In [47]:
days.rename(columns={'요일': '요일구분'}, inplace=True)
days = days[(days['호선'] == 1) | (days['호선'] == 2) | (days['호선'] == 3) | (days['호선'] == 4)]
days

,호선,역명,요일구분,비율
0,1,동대문,금,1.061534
1,1,동대문,목,1.020850
2,1,동대문,수,1.026641
3,1,동대문,월,1.005251
4,1,동대문,일,0.873151
...,...,...,...,...
835,4,회현(남대문시장),수,1.138622
836,4,회현(남대문시장),월,1.091271
837,4,회현(남대문시장),일,0.453112
838,4,회현(남대문시장),토,0.888779


In [48]:
merged_df = merged_df.replace({'요일구분' : '일요일'}, '일')
merged_df = merged_df.replace({'요일구분' : '토요일'}, '토')
merged_df.요일구분.value_counts()

요일구분
평일    33120
토     33120
일     33120
Name: count, dtype: int64

In [49]:
weekday = merged_df[merged_df['요일구분'] == '평일'].copy()
# weekday
weeks = ['월', '화', '수', '목', '금']
result_df = pd.concat([weekday.assign(요일구분=요일) for 요일 in weeks], ignore_index=True)
result_df

,요일구분,호선,역명,상하구분,시간대,값,사용월,비율,월혼잡도
0,월,1,종각,상선,05시~06시,9.0,1,1.026593,9.239336
1,월,1,종각,상선,05시~06시,9.0,2,0.897782,8.080034
2,월,1,종각,상선,05시~06시,9.0,3,1.027239,9.245150
3,월,1,종각,상선,05시~06시,9.0,4,0.987545,8.887901
4,월,1,종각,상선,05시~06시,9.0,5,1.014592,9.131327
...,...,...,...,...,...,...,...,...,...
165595,금,4,남태령,하선,24시~01시,19.0,8,1.193308,22.672848
165596,금,4,남태령,하선,24시~01시,19.0,9,1.096531,20.834094
165597,금,4,남태령,하선,24시~01시,19.0,10,0.997312,18.948929
165598,금,4,남태령,하선,24시~01시,19.0,11,1.014522,19.275927


In [50]:
result_df.tail(30)

,요일구분,호선,역명,상하구분,시간대,값,사용월,비율,월혼잡도
165570,금,4,남태령,하선,23시~24시,60.0,7,1.074550,64.473019
165571,금,4,남태령,하선,23시~24시,60.0,8,1.193308,71.598467
165572,금,4,남태령,하선,23시~24시,60.0,9,1.096531,65.791875
165573,금,4,남태령,하선,23시~24시,60.0,10,0.997312,59.838724
165574,금,4,남태령,하선,23시~24시,60.0,11,1.014522,60.871347
165575,금,4,남태령,하선,23시~24시,60.0,12,0.987340,59.240404
165576,금,4,남태령,상선,24시~01시,9.0,1,0.740849,6.667643
165577,금,4,남태령,상선,24시~01시,9.0,2,0.750525,6.754722
165578,금,4,남태령,상선,24시~01시,9.0,3,1.019584,9.176256
165579,금,4,남태령,상선,24시~01시,9.0,4,1.069612,9.626507


In [51]:
result_df.역명.value_counts()

역명
종각       2400
금호       2400
매봉       2400
양재       2400
남부터미널    2400
         ... 
용답       2400
신답       2400
용두       2400
도림천      2400
남태령      2400
Name: count, Length: 69, dtype: int64

In [52]:
fin = pd.concat([result_df, merged_df[merged_df['요일구분'].isin(['토', '일'])]], ignore_index=True)
fin.요일구분.value_counts()

요일구분
월    33120
화    33120
수    33120
목    33120
금    33120
토    33120
일    33120
Name: count, dtype: int64

In [53]:
fin['호선'] = fin['호선'].astype(int)
merged_fin = fin.merge(days, on=['호선', '역명', '요일구분'], how='inner')
merged_fin

,요일구분,호선,역명,상하구분,시간대,값,사용월,비율_x,월혼잡도,비율_y
0,월,1,종각,상선,05시~06시,9.0,1,1.026593,9.239336,1.130268
1,월,1,종각,상선,05시~06시,9.0,2,0.897782,8.080034,1.130268
2,월,1,종각,상선,05시~06시,9.0,3,1.027239,9.245150,1.130268
3,월,1,종각,상선,05시~06시,9.0,4,0.987545,8.887901,1.130268
4,월,1,종각,상선,05시~06시,9.0,5,1.014592,9.131327,1.130268
...,...,...,...,...,...,...,...,...,...,...
178075,일,4,남태령,하선,24시~01시,11.0,8,1.193308,13.126386,0.627717
178076,일,4,남태령,하선,24시~01시,11.0,9,1.096531,12.061844,0.627717
178077,일,4,남태령,하선,24시~01시,11.0,10,0.997312,10.970433,0.627717
178078,일,4,남태령,하선,24시~01시,11.0,11,1.014522,11.159747,0.627717


In [54]:
merged_fin.역명.value_counts()

역명
종각         3360
지축         3360
녹번         3360
홍제         3360
무악재        3360
독립문        3360
안국         3360
동대입구       3360
금호         3360
압구정        3360
신사         3360
잠원         3360
매봉         3360
대치         3360
학여울        3360
대청         3360
일원         3360
경찰병원       3360
당고개        3360
상계         3360
쌍문         3360
길음         3360
혜화         3360
명동         3360
신용산        3360
구파발        3360
신정네거리      3360
종로5가       3360
양천구청       3360
제기동        3360
상왕십리       3360
한양대        3360
뚝섬         3360
성수         3360
건대입구       3360
종합운동장      3360
역삼         3360
강남         3360
서초         3360
방배         3360
봉천         3360
신림         3360
신대방        3360
구로디지털단지    3360
문래         3360
당산         3360
홍대입구       3360
신촌         3360
아현         3360
용답         3360
신답         3360
도림천        3360
남태령        3360
Name: count, dtype: int64

In [55]:
merged_fin['최종혼잡도'] = merged_fin['월혼잡도'] * merged_fin['비율_y']
merged_fin

,요일구분,호선,역명,상하구분,시간대,값,사용월,비율_x,월혼잡도,비율_y,최종혼잡도
0,월,1,종각,상선,05시~06시,9.0,1,1.026593,9.239336,1.130268,10.442925
1,월,1,종각,상선,05시~06시,9.0,2,0.897782,8.080034,1.130268,9.132603
2,월,1,종각,상선,05시~06시,9.0,3,1.027239,9.245150,1.130268,10.449497
3,월,1,종각,상선,05시~06시,9.0,4,0.987545,8.887901,1.130268,10.045709
4,월,1,종각,상선,05시~06시,9.0,5,1.014592,9.131327,1.130268,10.320846
...,...,...,...,...,...,...,...,...,...,...,...
178075,일,4,남태령,하선,24시~01시,11.0,8,1.193308,13.126386,0.627717,8.239655
178076,일,4,남태령,하선,24시~01시,11.0,9,1.096531,12.061844,0.627717,7.571424
178077,일,4,남태령,하선,24시~01시,11.0,10,0.997312,10.970433,0.627717,6.886327
178078,일,4,남태령,하선,24시~01시,11.0,11,1.014522,11.159747,0.627717,7.005162


In [56]:
merged_fin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178080 entries, 0 to 178079
Data columns (total 11 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   요일구분    178080 non-null  object 
 1   호선      178080 non-null  int32  
 2   역명      178080 non-null  object 
 3   상하구분    178080 non-null  object 
 4   시간대     178080 non-null  object 
 5   값       178080 non-null  float64
 6   사용월     178080 non-null  int64  
 7   비율_x    178080 non-null  float64
 8   월혼잡도    178080 non-null  float64
 9   비율_y    178080 non-null  float64
 10  최종혼잡도   178080 non-null  float64
dtypes: float64(5), int32(1), int64(1), object(4)
memory usage: 14.3+ MB


In [57]:
merged_fin.isna().sum()

요일구분     0
호선       0
역명       0
상하구분     0
시간대      0
값        0
사용월      0
비율_x     0
월혼잡도     0
비율_y     0
최종혼잡도    0
dtype: int64

In [60]:
zero_ratio_data = merged_fin[(merged_fin['월혼잡도'] == 0) & (merged_fin['최종혼잡도'] == 0)]
zero_ratio_data.역명.value_counts()

역명
당고개        1764
쌍문          132
경찰병원        132
상계          108
독립문          48
도림천          48
양천구청         48
신정네거리        48
지축           48
구파발          48
녹번           48
홍제           48
무악재          48
안국           48
신용산          48
용답           48
금호           48
압구정          48
신사           48
잠원           48
길음           48
혜화           48
명동           48
동대입구         48
신답           48
아현           48
서초           48
상왕십리         48
한양대          48
뚝섬           48
성수           48
건대입구         48
신촌           48
역삼           48
강남           48
종합운동장        48
방배           48
봉천           48
신림           48
신대방          48
구로디지털단지      48
문래           48
당산           48
홍대입구         48
매봉           36
대치           36
학여울          36
대청           24
일원           24
제기동          24
남태령          24
Name: count, dtype: int64

In [53]:
zero_ratio_data.to_csv("0최종혼잡도,0월혼잡도(최종).csv", index=False)

### 일부 역에서 월 혼잡도, 최종 혼잡도 0이 나오는 이유
- 해당 시간대의 혼잡도가 0으로 기록되어 있기 때문.

In [61]:
merged_fin.to_csv("df_11_merged_fin(최종).csv", index=False)